In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# import torch
# import torch.nn as nn
# from torch.autograd import Variable
# import torch.optim as optim
# import torch.nn.functional as F
# import torch.nn as nn


In [ ]:
df = pd.read_csv('./baseline_df.csv')
print (df.shape)
print(df.columns.values)
df.head()


In [ ]:
# Looks like Charlotte has done this already.
df2 = df[df['land_mask']==1]

# Drop these columns that don't matter.
df2.drop(['pixel_id', 'land_mask'], axis=1, inplace=True)

# We ignore pixels with no agriculture, so 0 or missing calories per ha.
df2['calories_per_ha'] = df2['calories_per_ha'].replace({np.nan: 0})
df2 = df2[df2['calories_per_ha'] != 0]

# Replace all "I don't know" values with NaN, then nuke all rows with a NaN.
df2['slope'] = df2['slope'].replace({0: np.nan})  # 143 NaN in 'slope' variable, and 0 means vertical...

# 255 is the "missing value" value.
for soil_var in ['workability_index', 'toxicity_index', 'rooting_conditions_index', 'oxygen_availability_index',
                 'nutrient_retention_index', 'nutrient_availability_index', 'excess_salts_index', 
                 'protected_areas_index']:
    df2[soil_var] = df2[soil_var].replace({255: np.nan})

df2.drop(['calories_per_ha'], axis=1, inplace=True)
    
# Drop all rows with a NaN
df2 = df2.dropna()


In [ ]:
print(df2['log_calories_per_ha'].describe())
fig, ax = plt.subplots()
sns.distplot(df2['log_calories_per_ha'], ax=ax)

In [ ]:
# Normalizes a column between -1 and 1.
def normalize_col(df, col, xmin=None, xmax=None):
    if not xmin: xmin = df[col].min()
    if not xmax: xmax = df[col].max()
    df[col] = 2 * ((df[col] - xmin) / (xmax - xmin)) - 1
    # no return value, it's modified in place in df.


In [ ]:

for col in ['gdp_per_capita']:
    df2[col] = df2[col].apply(lambda x: np.log(x) if x != 0 else 0)
    df2.rename(columns = {col:'log_{}'.format(col)}, inplace = True)

# Slope: 90 => 0 is horizontal. Slope is now in radians.
df2['slope'] = df2['slope'].apply(lambda x:np.radians(-x))


# We normalize all column except the *index ones.

for col in df2:
    if col=='log_calories_per_ha': continue
    if '_index' in col: continue
    normalize_col(df2, col)

normalize_col(df2, 'log_calories_per_ha', 21.0, 24.0)  # Handcrafted.


# Move log_calories_per_ha to the front.
cols = df2.columns.tolist()
cols.insert(0, cols.pop(cols.index('log_calories_per_ha')))
df2 = df2.reindex(columns= cols)




In [ ]:
for col in df2:
    if 'index' in col: continue
    fig, ax = plt.subplots()
    sns.distplot(df2[col], ax=ax)

In [ ]:
print(df2['log_calories_per_ha'].describe())
fig, ax = plt.subplots()
sns.distplot(df2['log_calories_per_ha'], ax=ax)

print(df2['slope'].describe())
fig, ax = plt.subplots()
sns.distplot(df2['slope'], ax=ax)

In [ ]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [ ]:
print(df2.shape)
print(df2.columns.values)
df2.head()

In [ ]:
df2.to_csv('./data_for_model.csv', index=False)

In [ ]:
# Get back some memory.
del df
del df2


# Running the model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [ ]:
df = pd.read_csv('./data_for_model.csv')

In [ ]:
df.head()

In [ ]:
from collections import Counter

def find_exact_clusters(df):
    clusters = dict()   # Each entry is key => [list of log_ca for records with this key]
    for index, row in df.iterrows():
        log_ca = row[0]   # Horrible HACK: log_calories_per_ha is the FIRST entry.
        row2 = []
        for cell in row[1:]:    # Horrible HACK: and the actual input are the last n-1 columns.
            row2.append('{:.1f}'.format(cell))
        key = '_'.join(row2)
        if key not in clusters:
            clusters[key] = [log_ca]
        else:
            clusters[key].append(log_ca)
        
    sorted_by_value = sorted(clusters.items(), key=lambda kv: len(kv[1]), reverse=True)
    n = 0
    for kv in sorted_by_value:
        n += 1
        if n>100: break
        key = kv[0]
        values = kv[1]
        vmin, vmax, vave = min(values), max(values), sum(values)/len(values)
        ratio = np.exp(vmax-vmin)  # max(cal)/min(cal)
        print('{} \n\t min={:.3f} max={:.3f} ave={:.3f}, ratio={:.1f}, size={}\n'.format(
            key, min(values), max(values), sum(values)/len(values), ratio, len(values)))

#     for val,ct in clusters.most_common()[:100]:
#         print('{} clusters'.format(len(clusters)))
#         print('{}\t{}'.format(ct, val))

find_exact_clusters(df)

# Round to {:.0f}: 96383 clusters, top has 3216 entries
# Round to {:.1f}: 477184 clusters, top has 96 entries
# Round to {:.2f}: 579469 clusters, top has 2 entries
# Round to {:.3f}: 579524 clusters, top has 1 entry



In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn


In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        #m.weight.data.fill_(0.01)
        #m.bias.data.fill_(0.01)
        m.weight.data.uniform_()
        m.bias.data.uniform_()

# Create a model with Sequential
def create_model_2_layers(input_size, hidden_size, output_size, non_linearity = nn.Sigmoid()):
    model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            non_linearity,
            nn.Linear(hidden_size, hidden_size),
            non_linearity,
            nn.Linear(hidden_size, output_size)
            )
    model.apply(init_weights)
    return model

def create_model_3_layers(input_size, h1, h2, h3, output_size, non_linearity = nn.Sigmoid()):
    model = nn.Sequential(
            nn.Linear(input_size, h1),
            non_linearity,
            nn.Linear(h1, h2),
            non_linearity,
            nn.Linear(h2, h3),
            non_linearity,
            nn.Linear(h3, output_size)
            )
    model.apply(init_weights)
    return model

def create_model_4_layers(input_size, h1, h2, h3, h4, output_size, non_linearity = nn.Sigmoid()):
    model = nn.Sequential(
            nn.Linear(input_size, h1),
            non_linearity,
            nn.Linear(h1, h2),
            non_linearity,
            nn.Linear(h2, h3),
            non_linearity,
            nn.Linear(h3, h4),
            non_linearity,
            nn.Linear(h4, output_size)
            )
    model.apply(init_weights)
    return model


In [ ]:
def training_step(model, input_tensor, label_tensor, loss_function, optimizer):
    # Forward propagation.
    output = model(input_tensor)
    # Compute the loss.
    loss = loss_function(output, label_tensor)
    # Zero the gradients.
    model.zero_grad()
    # Compute the gradients.
    loss.backward()
    # Adjust system along negative error gradient
    optimizer.step()
    return output, loss.item()


def validation_step(model, input_tensor, label_tensor, loss_function):
    output = model(input_tensor)
    loss = loss_function(output, label_tensor)
    return output, loss.item()


def R2(y, y_hat):  # y are the labels, y_hat are the predictions.
    y = np.array(y)
    y_hat = np.array(y_hat)
    SSE = (y-y_hat)*(y-y_hat)
    y_mean = y.mean()
    SST = (y-y_mean)*(y-y_mean)
    return 1 - (SSE.sum() / SST.sum())


def train_model(model, df, training_epochs = 10, optimizer=None):
    loss_f = nn.MSELoss()
    if not optimizer: optimizer = optim.Adam(model.parameters(), lr = 3e-4, weight_decay = 0.001)
    
    print_every = 10000
    # Convert to tensors once. *****
    nb_rows = df.shape[0]
    nb_training_rows = nb_rows * 0.9
    
    for epoch in range(training_epochs):
        print("Epoch: {} - we reset the accumulated loss".format(epoch+1))
        loss_accum = 0
        num_its = 0
        valid_loss_accum = 0
        valid_num_its = 0
        ys = []    # Labels
        preds = [] # Predictions
        for index, row in df.iterrows():
            row_as_list = row.tolist()
            label = row_as_list[0]
            label_tensor = torch.as_tensor([label])
            all_other_columns = row_as_list[1:]
            input_tensor = torch.tensor(all_other_columns)
            
            if index < nb_training_rows:    # Training the model.
                # This only works because the label is the first column!!!
                output, loss = training_step(model, input_tensor, label_tensor, loss_f, optimizer)
                loss_accum += loss
                num_its += 1
                if num_its % print_every == 0 and num_its != 0:
                    print("Loss (MSE): {:.3f}".format(loss_accum / num_its))
            else:                           # Validating the model.
                output, loss = validation_step(model, input_tensor, label_tensor, loss_f)
                valid_loss_accum += loss
                valid_num_its += 1
                pred = output.data.numpy()[0]
                
                ys.append(label)
                preds.append(pred)
#                 if valid_num_its % 100 == 0:
#                     print('output={:.3f} label={:.3f} loss={:.3f}'.format(pred, label, loss))
                if valid_num_its % print_every == 0 and valid_num_its != 0:
                    print("Validation loss (MSE): {:.3f}".format(valid_loss_accum / valid_num_its))
                    print('R2={}'.format(R2(ys, preds)))
                    
                
                
    # Output model to file.
    torch.save(model, 'model_pour_charlotte')

# # Run the model
# def run_model(model, input_tensor):
#     return torch.nn.functional.softmax(model(input_tensor))

# input_data = pd.read_csv('data/nutanix-test.tsv', sep='\t')
# probs = [None] * len(input_data)
# for entry in test_data:
#     probs[entry['src_line']] = float(run_model(ffnn, companyToTensor(entry))[1])
# output_data = input_data.assign(Fitness=pd.Series(probs).values)
# output_data = output_data.sort_values(by=['Fitness'], ascending=False)
# output_data.to_csv('data/nutanix-test-classified.tsv', sep='\t')

In [ ]:
n_inputs = df.shape[1] - 1   # So far, 59 - 1 (label) = 58
# 2 layers
# model = create_model_2_layers(n_inputs, 5, 1)   # Validation loss (MSE): 0.051  R2=-0.0038
model = create_model_2_layers(n_inputs, 20, 1)  # Loss (MSE): 0.051
# model = create_model(n_inputs, 5, 1)   # MSE loss = 1.56 after 3 epochs.  (R2=0.173 after 3 epochs)
# model = create_model(n_inputs, 10, 1)   # MSE loss = 1.56 after 3 epochs.
# model = create_model(n_inputs, 20, 1)   # MSE loss = 1.55, val loss = 1.482 after 10 epochs
# model = create_model(n_inputs, 50, 1)   # MSE loss = , val loss =  after 10 epochs

# 3 layers
# model = create_model(n_inputs, 5, 1)   # MSE loss = , val loss = , R2 =   after 10 epochs
# model = create_model(n_inputs, 10, 1)   # MSE loss = 1.463, val loss = 1.448 after 10 epochs
# model = create_model(n_inputs, 20, 1)   # MSE loss = 1.555, val loss = 1.616 after 2 epochs


# All inputs normalized in -1..1, log_calories_per_ha, 2 layers
# model = create_model_2_layers(n_inputs, 50, 1)  # MSE loss = 1.120, val loss = 1.072, R2=0.40 after 10 epochs
# model = create_model_2_layers(n_inputs, 100, 1)   # MSE loss = 1.159, val loss = 1.087, R2=0.394 after 3 epochs
# model = create_model_2_layers(n_inputs, 200, 1)   # MSE loss = , val loss = , R2= after  epochs

# Every field included
# model = create_model_2_layers(n_inputs, 50, 1)   # MSE loss = 1.123, val loss = 1.041, R2= 0.402 after 10 epochs
# Removing lat/lon
# model = create_model_2_layers(n_inputs, 50, 1)   # MSE loss = 1.176, val loss = 1.106, R2= 0.365 after 10 epochs
# Removing (only) climatezone_*
# model = create_model_2_layers(n_inputs, 50, 1)   # MSE loss = 1.147, val loss = 1.061, R2= 0.391 after 10 epochs
# Removing log_min_to_market
# model = create_model_2_layers(n_inputs, 50, 1)   # MSE loss = 1.134, val loss = 1.050, R2= 0.397 after 10 epochs
# Removing gdp and log_gdp_per_capita
# model = create_model_2_layers(n_inputs, 50, 1)   # MSE loss = 1.147, val loss = 1.065, R2= 0.384 after 10 epochs

# All fields, *index have been encoded as 1-hot.
# model = create_model_2_layers(n_inputs, 50, 1)   # MSE loss = 1.107, val loss = 1.108, R2= 0.406 after 10 epochs
# Same, slope has been stretched.
# model = create_model_2_layers(n_inputs, 50, 1)   # MSE loss = 1.125, val loss = 1.110, R2= 0.387 after 10 epochs

# Back to regular slope (so back to normal), but we go nuts on width.
# model = create_model_2_layers(n_inputs, 300, 1)
                                                   # MSE loss = 1.165, val loss = 1.008, R2= 0.414 after 10 epochs

# model = create_model_3_layers(n_inputs, 300, 100, 30, 1)
                                                   # MSE loss = 1.131, val loss = 0.975, R2= 0.433 after 20 epochs
    
# model = create_model_3_layers(n_inputs, 10, 10, 10, 1)       # val=1.00, R2=0.415 after 6 epochs

# model = create_model_4_layers(n_inputs, 10, 10, 10, 10, 1)     # val=0.989, R2=0.425, 15 epochs
# model = create_model_4_layers(n_inputs, 20, 20, 20, 10, 1)     # val=0.970, R2=0.436, 20 epochs
# model = create_model_4_layers(n_inputs, 50, 50, 20, 10, 1)     # val=0.973, R2=0.434, 20 epochs

# model = create_model_4_layers(n_inputs, 10, 10, 10, 10, 1)     # val=0.989, R2=0.425, 100 epochs


# model = create_model_4_layers(n_inputs, 20, 20, 20, 10, 1, non_linearity = nn.ReLU())     
                                                                # val=0.910, R2=0.471, 100 epochs

# model = create_model_4_layers(n_inputs, 50, 50, 50, 10, 1, non_linearity = nn.ReLU())     
                                                                # val=1.012, R2=0.412, 7 epochs, weight_decay=0.01

# model = create_model_4_layers(n_inputs, 50, 50, 50, 10, 1, non_linearity = nn.ReLU())     
                                                                # val=1.121, R2=0.349, 88 epochs, weight_decay=0.1

model


In [ ]:
optimizer = optim.Adam(model.parameters(), lr = 3e-4, weight_decay = 0.1)
train_model(model, df, training_epochs=100, optimizer=optimizer)


In [ ]:
for col in df:
    fig, ax = plt.subplots()
    sns.distplot(df[col], ax=ax)